## First, install the Python dependencies

In [1]:
!pip install --upgrade pip && pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 93.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 193.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 721.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 305.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 427.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 182.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 304.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 449.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 261.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 448.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 195.8 MB/s eta 0:00:

## Initialize the feature store

In [3]:
!feast -c $HOME/feast-credit-score-local-tutorial/feature_repo apply

No project found in the repository. Using project name credit_scoring_local defined in feature_store.yaml
Applying changes for project credit_scoring_local
/opt/app-root/lib64/python3.11/site-packages/feast/feature_store.py:581: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
Deploying infrastructure for credit_history
Deploying infrastructure for zipcode_features


## Materialize the data into the online feature store

In [5]:
!feast -c $HOME/feast-credit-score-local-tutorial/feature_repo materialize 1990-01-01T00:00:00 $(date -u +"%Y-%m-%dT%H:%M:%S")

Materializing 2 feature views from 1990-01-01 00:00:00+00:00 to 2025-03-14 19:24:09+00:00 into the redis online store.

credit_history:
100%|██████████████████████████████████████████████████████| 28633/28633 [00:02<00:00, 11834.60it/s]
zipcode_features:
100%|██████████████████████████████████████████████████████| 28844/28844 [00:01<00:00, 14910.20it/s]


In [6]:
!python run.py

Starting model training...
Model has been trained successfully
Submitting a prediction to the model with data {'zipcode': [76104], 'dob_ssn': ['19630621_4278'], 'person_age': [133], 'person_income': [59000], 'person_home_ownership': ['RENT'], 'person_emp_length': [123.0], 'loan_intent': ['PERSONAL'], 'loan_amnt': [35000], 'loan_int_rate': [16.02]}
Data returned from online store for request:
         dob_ssn  total_debt_due  credit_card_due  mortgage_due  \
0  19630621_4278        473071.0             3343        378847   

   student_loan_due  vehicle_loan_due  loan_amnt  
0             44375             11506      35000  
Loan rejected!


## Explore some entities, etc. on Feast

In [7]:
!feast -c $HOME/feast-credit-score-local-tutorial/feature_repo data-sources list

NAME              CLASS
application_data  <class 'feast.data_source.RequestSource'>
Zipcode source    <class 'feast.infra.offline_stores.file_source.FileSource'>
Credit history    <class 'feast.infra.offline_stores.file_source.FileSource'>


In [8]:
!feast -c $HOME/feast-credit-score-local-tutorial/feature_repo entities list

NAME     DESCRIPTION                                                   TYPE
zipcode                                                                ValueType.INT64
dob_ssn  Date of birth and last four digits of social security number  ValueType.STRING


In [9]:
!feast -c $HOME/feast-credit-score-local-tutorial/feature_repo feature-views list

NAME              ENTITIES     TYPE
credit_history    {'dob_ssn'}  FeatureView
zipcode_features  {'zipcode'}  FeatureView
total_debt_calc   {'dob_ssn'}  OnDemandFeatureView


In [10]:
!feast -c $HOME/feast-credit-score-local-tutorial/feature_repo on-demand-feature-views describe total_debt_calc

spec:
  name: total_debt_calc
  features:
  - name: total_debt_due
    valueType: DOUBLE
  sources:
    application_data:
      requestDataSource:
        type: REQUEST_SOURCE
        requestDataOptions:
          schema:
          - name: loan_amnt
            valueType: INT64
        name: application_data
    credit_history:
      featureViewProjection:
        featureViewName: credit_history
        featureColumns:
        - name: credit_card_due
          valueType: INT64
        - name: mortgage_due
          valueType: INT64
        - name: student_loan_due
          valueType: INT64
        - name: vehicle_loan_due
          valueType: INT64
        - name: hard_pulls
          valueType: INT64
        - name: missed_payments_2y
          valueType: INT64
        - name: missed_payments_1y
          valueType: INT64
        - name: missed_payments_6m
          valueType: INT64
        - name: bankruptcies
          valueType: INT64
        timestampField: created_timestamp
    